In [1]:
import numpy as np
import pandas as pd
import altair as alt

# Save a vega-lite spec and a PNG blob for each plot in the notebook
alt.renderers.enable("mimetype")
# Handle large data sets without embedding them in the notebook
alt.data_transformers.enable("data_server");

In [2]:
stinson2019 = pd.read_pickle("../data/stinson2019/stinson2019_df")
HF_weather = pd.read_csv('../data/HF_weather/raw/hf001-08-hourly-m.csv', parse_dates=['datetime'])

/home/steffen/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (3,5,7,9,11,13,15,17,19,21,23,25,27,29) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
# stinson2019_df["jd"] = pd.DatetimeIndex(stinson2019_df["date"]).dayofyear
# stinson2019_df["year_int"] = pd.DatetimeIndex(stinson2019_df["year"]).year
# stinson2019_df["sugar_wt"] = stinson2019_df["sugar"]/100 * stinson2019_df["sap_wt"]
# stinson2019_df["yearly_sap"] = np.nan
# stinson2019_df["yearly_sugar"] = np.nan
# stinson2019_df['tree_tap'] = stinson2019_df['tree']+ '_' + stinson2019_df['tap']
# stinson2019_df = stinson2019_df.sort_values(by="date")

# abs_measurement = 0

# for year in stinson2019_df["year_int"].unique():
#     year_df = stinson2019_df[stinson2019_df["year_int"] == year]

#     for site in stinson2019_df[stinson2019_df["year_int"] == year]["site"].unique():
#         for tree in stinson2019_df[
#             (stinson2019_df["year_int"] == year) & (stinson2019_df["site"] == site)
#         ]["tree"].unique():
#             for tap in stinson2019_df[
#                 (stinson2019_df["year_int"] == year)
#                 & (stinson2019_df["site"] == site)
#                 & (stinson2019_df["tree"] == tree)
#             ]["tap"].unique():
#                 tap_yr_index = stinson2019_df.index[
#                     (stinson2019_df["year_int"] == year)
#                     & (stinson2019_df["site"] == site)
#                     & (stinson2019_df["tree"] == tree)
#                     & (stinson2019_df["tap"] == tap)
#                 ].tolist()
#                 yearly_sap = 0
#                 yearly_sugar = 0
#                 for measurement in tap_yr_index:
#                     yearly_sap = np.nansum(
#                         [yearly_sap, stinson2019_df.loc[measurement, "sap_wt"]]
#                     )
#                     yearly_sugar = np.nansum(
#                         [
#                             yearly_sugar,
#                             stinson2019_df.loc[measurement, "sap_wt"]
#                             * stinson2019_df.loc[measurement, "sugar"]/100,
#                         ]
#                     )
#                     stinson2019_df.loc[measurement, "yearly_sap"] = yearly_sap
#                     stinson2019_df.loc[measurement, "yearly_sugar"] = yearly_sugar


# #     yearly_sap = 0
# #     yearly_sugar = 0
# #     for measurement in list(range(len(year_df))):
# #         yearly_sap+=stinson2019_df.iloc[abs_measurement,6]
# #         yearly_sugar+=stinson2019_df.iloc[abs_measurement,6]*stinson2019_df.iloc[abs_measurement,5]
# #         stinson2019_df.iloc[abs_measurement,13]=yearly_sap
# #         stinson2019_df.iloc[abs_measurement,14]=yearly_sugar
# #         abs_measurement+=1

In [25]:
stinson2019 = stinson2019.sort_values(["site", "tree", "tap", "date"])

# Create unique record ids for each entry
stinson2019["record_id"] = (
    + stinson2019["tree"]
    + "_"
    + stinson2019["tap"]
    + "_"
    + pd.DatetimeIndex(stinson2019["year"]).year.astype(str)
    + "_"
)

for tapyear in stinson2019["record_id"].unique():
    stinson2019.loc[stinson2019["record_id"] == tapyear, "record_id"] += [
        str(i)
        for i in range(stinson2019[stinson2019["record_id"] == tapyear].shape[0])
    ]
    
stinson2019['tap_id'] = stinson2019['tree']+stinson2019['tap']


# normalize data tables
tap_records = stinson2019[['record_id', 'tap_id']].set_index('record_id')
sap = stinson2019[['record_id', 'sap_wt']].rename(columns={"sap_wt": "sap"}).set_index('record_id')
sugar = stinson2019[['record_id', 'sugar']].set_index('record_id')
dates = stinson2019[['record_id', 'date']].set_index('record_id')
dates.loc[:,'date'] = pd.to_datetime(dates['date'])
tap_tree = stinson2019[['tap_id','tree']].drop_duplicates().set_index('tap_id')
species = stinson2019[['tree','species']].drop_duplicates().set_index('tree')
site = stinson2019[['tree','site']].drop_duplicates().set_index('tree')

In [10]:
stinson2019.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9929 entries, 5299 to 7913
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   site            9929 non-null   object        
 1   date            9929 non-null   datetime64[ns]
 2   tree            9929 non-null   object        
 3   species         9929 non-null   object        
 4   tap             9929 non-null   object        
 5   sugar           8139 non-null   float64       
 6   sap_wt          9296 non-null   float64       
 7   year            9929 non-null   datetime64[ns]
 8   site_id         9929 non-null   object        
 9   tree_id         9929 non-null   object        
 10  tree_record_id  9929 non-null   object        
 11  tap_record_id   9929 non-null   object        
 12  record_id       9929 non-null   object        
 13  tap_id          9929 non-null   object        
dtypes: datetime64[ns](2), float64(2), object(10)
memory u

In [41]:
dates[pd.DatetimeIndex(dates['date']).year.isin(['2014'])]

,date
record_id,
DOF1_A_2014_0,2014-03-11
DOF1_A_2014_1,2014-03-17
DOF1_A_2014_2,2014-03-30
DOF1_A_2014_3,2014-03-31
DOF1_A_2014_4,2014-04-01
...,...
SM9_B_2014_6,2014-03-15
SM9_B_2014_7,2014-03-19
SM9_B_2014_8,2014-03-21


In [40]:
dates['date'].year

AttributeError: 'Series' object has no attribute 'year'

In [92]:
years = [2014, 2015]
tapid = "DOF1A"

df = (
    tap_records.join(tap_tree[tap_tree.index == tapid], how="right", on="tap_id")
    .join(sap, how="left")
    .join(sugar, how="left")
    .join(dates[pd.DatetimeIndex(dates["date"]).year.isin(years)], how="inner")
)
df["year"] = pd.DatetimeIndex(df["date"]).year
df["jd"] = pd.DatetimeIndex(df["date"]).dayofyear

# Assumption: missing sugar content should be filled with mean sugar content
df['sugarwt'] = df.sap*df.sugar.fillna(value = df.sugar.mean())

# Assumption: missing sap values should be replaced with zeros
df["cum_sap"] = df.sap.fillna(value=0).cumsum()
df["cum_sugarwt"] = df.sugarwt.fillna(value=0).cumsum()

for year in years:
    df_year = df[df['year']==year]
    num_wks = df_year['jd'].max()//7
    wkday1 = list(range(1,num_wks*7,7))
    for week in range(num_wks):
        print(week)
#     df.groupby(df_year]['date']).sum()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [90]:
df

,tap_id,tree,sap,sugar,date,year,jd,sugarwt,cum_sap,cum_sugarwt
record_id,,,,,,,,,,
DOF1_A_2014_0,DOF1A,DOF1,0.03,1.2,2014-03-11,2014,70,0.0360,0.03,0.0360
DOF1_A_2014_1,DOF1A,DOF1,0.01,NaN,2014-03-17,2014,76,0.0246,0.04,0.0606
DOF1_A_2014_2,DOF1A,DOF1,NaN,NaN,2014-03-30,2014,89,NaN,0.04,0.0606
DOF1_A_2014_3,DOF1A,DOF1,0.01,NaN,2014-03-31,2014,90,0.0246,0.05,0.0852
DOF1_A_2014_4,DOF1A,DOF1,0.67,2.2,2014-04-01,2014,91,1.4740,0.72,1.5592
DOF1_A_2014_5,DOF1A,DOF1,0.35,2.2,2014-04-02,2014,92,0.7700,1.07,2.3292
DOF1_A_2014_6,DOF1A,DOF1,1.11,2.1,2014-04-04,2014,94,2.3310,2.18,4.6602
DOF1_A_2014_7,DOF1A,DOF1,0.29,1.4,2014-04-05,2014,95,0.4060,2.47,5.0662
DOF1_A_2014_8,DOF1A,DOF1,NaN,NaN,2014-04-06,2014,96,NaN,2.47,5.0662


In [89]:
wkday1

[1, 8, 15, 22, 29, 36, 43, 50, 57, 64, 71, 78, 85, 92, 99]

In [ ]:
def get_weekly_data(location = 'all', tree= 'all',tap_id='all',years='all'):
    df = tap_records.join(tap_tree.iloc[[0]], how="right", on="tap_id").join(
        sap, how="left").join(
        sugar, how="left").join(
        dates, how="left")
    df['year'] = pd.DatetimeIndex(df['date']).year
    df['jd'] = pd.DatetimeIndex(df['date']).dayofyear
    for year in df['year']:
        df_year = df[df['year']==year]
        num_wks = df_year['jd'].max()//7
        weeks = list(range(0,num_wks,7))
        df.groupby(df_year]['date']).sum()
        harvard_sap.groupby([harvard_sap['year'], pd.cut(df_year['jd'], weeks)]).sum()
    if year == 'all':
        years = 
    
    weekly_df = 'tap_id', 'dates', 'year', 'week', 'c_sap', 'c_sugar', 'fr_thw', 'gdd'
    
    return weekly_df

In [7]:
harvard_sap = stinson2019_df[stinson2019_df['site_id']=='HF']
harvard_sap['jd'].max()

102

In [8]:
weeks = list(range(0,106,7))
harvard_sap.groupby([harvard_sap['year'], pd.cut(harvard_sap['jd'], weeks)]).sum()

sugar  sap_wt       jd  year_int  sugar_wt  yearly_sap  \
year       jd                                                                  
2012-01-01 (0, 7]       NaN     NaN      NaN       NaN       NaN         NaN   
           (7, 14]      NaN     NaN      NaN       NaN       NaN         NaN   
           (14, 21]     NaN     NaN      NaN       NaN       NaN         NaN   
           (21, 28]     NaN     NaN      NaN       NaN       NaN         NaN   
           (28, 35]     NaN     NaN      NaN       NaN       NaN         NaN   
...                     ...     ...      ...       ...       ...         ...   
2017-01-01 (70, 77]     NaN     NaN      NaN       NaN       NaN         NaN   
           (77, 84]   196.8  287.54   7632.0  193632.0   6.73874     4241.60   
           (84, 91]   498.2  823.22  20880.0  484080.0  18.96144    13784.15   
           (91, 98]   243.8  343.14  12300.0  266244.0   7.71448     9610.41   
           (98, 105]    NaN     NaN      NaN       NaN       NaN         NaN   

                      yearly_sugar  
year       jd                       
2012-01-01 (0, 7]              NaN  
           (7, 14]             NaN  
           (14, 21]            NaN  
           (21, 28]            NaN  
           (28, 35]            NaN  
...                            ...  
2017-01-01 (70, 77]            NaN  
           (77, 84]      103.54440  
           (84, 91]      332.57985  
           (91, 98]      231.82877  
           (98, 105]           NaN  

[90 rows x 7 columns]

In [9]:
pd.periodindex

AttributeError: module 'pandas' has no attribute 'periodindex'